To run this code, you first have to establish an anaconda environment, ensure you have the relevant interpreters/languages/libraries downloaded, and ensure that the names of the relevant .csv files on your comptuer/server match those on the code.

First, we will import the pandas data analytics library, as well as the pre and post SST raw survey data. We will also create a global `resuts` data frame to hold all of our metrics.

In [123]:
import pandas as pd
pre_sst_responses = pd.read_csv('C:\\Users\\arira\\OneDrive\\Desktop\\Better Work\\Nicaragua Project\\CSV Files\\Raw Data\\Pre SST questions_August 25, 2022_12.46.csv')
post_sst_responses = pd.read_csv('C:\\Users\\arira\\OneDrive\\Desktop\\Better Work\\Nicaragua Project\\CSV Files\\Raw Data\\Post SST questions_August 25, 2022_12.47.csv')
results = pd.DataFrame(columns = ['Metric','Value'])

In order to ensure that the pre and post SST data stays alligned and is analyzed uniformly, we need to do this analysis via two primary functions: a `clean` function that prepares both response sets for analysis, and an `analyze` function that takes these two cleaned data sets and calculates our desired metrics. 

First, we will define the `clean` function. In this function, we need to restructure the data slightly to allow for more flexible analysis.

In [124]:
def clean(df):

    df = df.fillna(0)

    for x in range(0,2):
        df.drop(x, axis=0, inplace=True)
    df = df.reset_index()
    df.drop('index',axis=1,inplace=True)

    for y in range(2,5):
        for x in range(len(df['Q9_1'])):
            df.loc[int(x),'Q9_1'] = int(df.loc[int(x),'Q9_1']) + y*int(df.loc[int(x), 'Q9_' + str(y)])
        df.drop('Q9_' + str(y), axis = 1, inplace=True)
    df.rename({'Q9_1': 'Q9'}, axis=1, inplace=True)

    for y in range(1,9):
        for x in range(len(df['Q3_9'])):
            if df.loc[x,'Q3_9'] == 1:
                df.loc[int(x),'Q3_'+ str(y)] = 1
    df.drop('Q3_9', axis=1, inplace=True)

    for z in range(1,7):
        for y in range(2,6):
            for x in range(len(df['Q4_' + str(z) + '_' + str(y)])):
                if df.loc[int(x),'Q4_' + str(z) + '_1'] == 0:
                    df.loc[int(x),'Q4_' + str(z) + '_1'] = int(df.loc[int(x),'Q4_' + str(z) + '_1']) + y*int(df.loc[int(x), 'Q4_' + str(z) + '_' + str(y)])
            df.drop('Q4_' + str(z) + '_' + str(y), axis = 1, inplace=True)
        df.rename({'Q4_' + str(z) + '_1': 'Q4_' + str(z)}, axis=1, inplace=True)

    df['Q8'] = df['Q8'].astype('int')
    df['StartDate'] = pd.to_datetime(df['StartDate'])

    return df

Next, we will create the `analyze` function. Due to its complexity, it will be composed of a few subfunctions where neccesary.

In [125]:
def analyze(pre_df, post_df):

    metrics = ['pre_total_respondents','post_total_respondents','pre_production_worker', 'pre_non_production_worker', 'pre_supervisor', 'pre_administrative/management','post_production_worker', 'post_non_production_worker', 'post_supervisor', 'post_administrative/management', 'pre_women', 'pre_men', 'pre_prefer_not_to_say', 'post_women','post_men', 'post_prefer_not_to_say', 'change_supervisory_responsibility_1', 'change_supervisory_responsibility_2', 'change_supervisory_responsibility_3', 'change_supervisory_responsibility_4', 'change_supervisory_responsibility_5', 'change_supervisory_responsibility_6', 'change_supervisory_responsibility_7', 'change_supervisory_responsibility_average','change_supervisory_responsibility_production_worker', 'change_supervisory_responsibility_non_production_worker', 'change_supervisory_responsibility_supervisor', 'change_supervisory_responsibility_administrative/management', 'pre_objective_identification_1', 'pre_objective_identification_2', 'pre_objective_identification_3', 'pre_objective_identification_4', 'pre_objective_identification_5', 'post_objective_identification_1', 'post_objective_identification_2', 'post_objective_identification_3', 'post_objective_identification_4', 'post_objective_identification_5','change_correct_objective_identification_production_worker', 'change_correct_objective_identification_non_production_worker', 'change_correct_objective_identification_supervisor', 'change_correct_objective_identification_administrative/management', 'change_correct_objective_identification_average', 'change_recognition_harrassment_1', 'change_recognition_harrassment_2', 'change_recognition_harrassment_3', 'change_recognition_harrassment_4', 'change_recognition_harrassment_5', 'change_recognition_harrassment_6', 'change_recognition_harrassment_7', 'change_recognition_harrassment_8', 'change_recognition_harrassment_average', 'change_recognition_harrassment_production_worker', 'change_recognition_harrassment_non_production_worker', 'change_recognition_harrassment_supervisor', 'change_recognition_harrassment_administrative/management', 'change_recognition_harrassment_women', 'change_recognition_harrassment_men', 'pre_practice_agreement_1_production_worker', 'pre_practice_agreement_2_production_worker','pre_practice_agreement_3_production_worker','pre_practice_agreement_4_production_worker','pre_practice_agreement_5_production_worker', 'pre_practice_agreement_1_non_production_worker', 'pre_practice_agreement_2_non_production_worker','pre_practice_agreement_3_non_production_worker','pre_practice_agreement_4_non_production_worker','pre_practice_agreement_5_non_production_worker', 'pre_practice_agreement_1_supervisor', 'pre_practice_agreement_2_supervisor','pre_practice_agreement_3_supervisor','pre_practice_agreement_4_supervisor','pre_practice_agreement_5_supervisor', 'pre_practice_agreement_1_administrative/management', 'pre_practice_agreement_2_administrative/management','pre_practice_agreement_3_administrative/management','pre_practice_agreement_4_administrative/management','pre_practice_agreement_5_administrative/management', 'post_practice_agreement_1_production_worker', 'post_practice_agreement_2_production_worker','post_practice_agreement_3_production_worker','post_practice_agreement_4_production_worker','post_practice_agreement_5_production_worker', 'post_practice_agreement_1_non_production_worker', 'post_practice_agreement_2_non_production_worker','post_practice_agreement_3_non_production_worker','post_practice_agreement_4_non_production_worker','post_practice_agreement_5_non_production_worker', 'post_practice_agreement_1_supervisor', 'post_practice_agreement_2_supervisor','post_practice_agreement_3_supervisor','post_practice_agreement_4_supervisor','post_practice_agreement_5_supervisor', 'post_practice_agreement_1_administrative/management', 'post_practice_agreement_2_administrative/management','post_practice_agreement_3_administrative/management','post_practice_agreement_4_administrative/management','post_practice_agreement_5_administrative/management', 'change_practice_agreement', 'pre_gender_discrimination', 'post_gender_discrimination', 'change_gender_discrimination','change_gender_discrimination_women','change_gender_discrimination_men', 'change_gender_discrimination_general', 'change_gender_discrimination_gender_specific']
    metrics = pd.Series(metrics)
    results['Metric'] = metrics
    results['Value'] = float(0)
    
    results.loc[0,'Value'] = len(pre_df.index)
    results.loc[1,'Value'] = len(post_df.index)

    pre_df_production_worker_only = pre_df.loc[pre_df['Q9'] == 1]
    pre_df_non_production_worker_only = pre_df.loc[pre_df['Q9'] == 2]
    pre_df_supervisor_only = pre_df.loc[pre_df['Q9'] == 3]
    pre_df_administrative_and_management_only = pre_df.loc[pre_df['Q9'] == 4]
    
    results.loc[2, 'Value'] = len(pre_df_production_worker_only.index)
    results.loc[3, 'Value'] = len(pre_df_non_production_worker_only.index)
    results.loc[4, 'Value'] = len(pre_df_supervisor_only.index)
    results.loc[5, 'Value'] = len(pre_df_administrative_and_management_only.index)

    post_df_production_worker_only = post_df.loc[post_df['Q9'] == 1]
    post_df_non_production_worker_only = post_df.loc[post_df['Q9'] == 2]
    post_df_supervisor_only = post_df.loc[post_df['Q9'] == 3]
    post_df_administrative_and_management_only = post_df.loc[post_df['Q9'] == 4]
    
    results.loc[6, 'Value'] = len(post_df_production_worker_only.index)
    results.loc[7, 'Value'] = len(post_df_non_production_worker_only.index)
    results.loc[8, 'Value'] = len(post_df_supervisor_only.index)
    results.loc[9, 'Value'] = len(post_df_administrative_and_management_only.index)

    pre_df_women_only = pre_df.loc[pre_df['Q8'] == 1]
    pre_df_men_only = pre_df.loc[pre_df['Q8'] == 2]
    pre_df_prefer_not_to_say_only = pre_df.loc[pre_df['Q8'] == 3]

    results.loc[10, 'Value'] = len(pre_df_women_only.index)
    results.loc[11, 'Value'] = len(pre_df_men_only.index)
    results.loc[12, 'Value'] = len(pre_df_prefer_not_to_say_only.index)

    post_df_women_only = post_df.loc[post_df['Q8'] == 1]
    post_df_men_only = post_df.loc[post_df['Q8'] == 2]
    post_df_prefer_not_to_say_only = post_df.loc[post_df['Q8'] == 3]

    results.loc[13, 'Value'] = len(post_df_women_only.index)
    results.loc[14, 'Value'] = len(post_df_men_only.index)
    results.loc[15, 'Value'] = len(post_df_prefer_not_to_say_only.index)
    
    pre_df_general = pre_df.loc[pre_df['StartDate'].dt.month == 5]
    pre_df_gender_specific = pre_df.loc[pre_df['StartDate'].dt.month == 4]

    post_df_general = post_df.loc[post_df['StartDate'].dt.month == 5]
    post_df_gender_specific = post_df.loc[post_df['StartDate'].dt.month == 4]

    total = 0
    for x in range(1,8):
        change = change_supervisory_responsibility(pre_df, post_df, 'Q1_'+str(x))
        results.loc[x+15, 'Value'] = round(change,2)
        total = total + change
    average = total/7
    results.loc[23, 'Value'] = round(average,2)

    total = 0
    for x in range(1,8):
        change = change_supervisory_responsibility(pre_df_production_worker_only, post_df_production_worker_only, 'Q1_'+str(x))
        total = total + change
    average = total/7
    results.loc[24, 'Value'] = round(average,2)

    total = 0
    for x in range(1,8):
        change = change_supervisory_responsibility(pre_df_non_production_worker_only, post_df_non_production_worker_only, 'Q1_'+str(x))
        total = total + change
    average = total/7
    results.loc[25, 'Value'] = round(average,2)

    total = 0
    for x in range(1,8):
        change = change_supervisory_responsibility(pre_df_supervisor_only, post_df_supervisor_only, 'Q1_'+str(x))
        total = total + change
    average = total/7
    results.loc[26, 'Value'] = round(average,2)

    total = 0
    for x in range(1,8):
        change = change_supervisory_responsibility(pre_df_administrative_and_management_only, post_df_administrative_and_management_only, 'Q1_'+str(x))
        total = total + change
    average = total/7
    results.loc[27, 'Value'] = round(average,2)
    
    count(pre_df, 'Q2', 5, 27)
    count(post_df, 'Q2', 5, 32)
    
    results.loc[38,'Value'] = round(change_correct_count(pre_df_production_worker_only,post_df_production_worker_only, 'Q2',5), 2)
    results.loc[39,'Value'] = round(change_correct_count(pre_df_non_production_worker_only,post_df_non_production_worker_only, 'Q2',5), 2)
    results.loc[40,'Value'] = round(change_correct_count(pre_df_supervisor_only,post_df_supervisor_only, 'Q2',5), 2)
    results.loc[41,'Value'] = round(change_correct_count(pre_df_administrative_and_management_only,post_df_administrative_and_management_only, 'Q2',5), 2)
    results.loc[42,'Value'] = round(change_correct_count(pre_df,post_df, 'Q2',5), 2)

    change_harrassment_recognition(pre_df,post_df,'Q3',1,8,42,False)
    change_harrassment_recognition(pre_df,post_df,'Q3',1,8,51,True)
    change_harrassment_recognition(pre_df_production_worker_only,post_df_production_worker_only,'Q3',1,8,52,True)
    change_harrassment_recognition(pre_df_non_production_worker_only,post_df_non_production_worker_only,'Q3',1,8,53,True)
    change_harrassment_recognition(pre_df_supervisor_only,post_df_supervisor_only,'Q3',1,8,54,True)
    change_harrassment_recognition(pre_df_administrative_and_management_only,post_df_administrative_and_management_only,'Q3',1,8,55,True)
    change_harrassment_recognition(pre_df_women_only,post_df_women_only,'Q3',1,8,56,True)
    change_harrassment_recognition(pre_df_men_only,post_df_women_only,'Q3',1,8,57,True)

    practice_agreement(pre_df_production_worker_only,'Q4',5,57)
    practice_agreement(pre_df_non_production_worker_only,'Q4',5,62)
    practice_agreement(pre_df_supervisor_only,'Q4',5,67)
    practice_agreement(pre_df_administrative_and_management_only,'Q4',5,72)
    
    practice_agreement(post_df_production_worker_only,'Q4',5,77)
    practice_agreement(post_df_non_production_worker_only,'Q4',5,82)
    practice_agreement(post_df_supervisor_only,'Q4',5,87)
    practice_agreement(post_df_administrative_and_management_only,'Q4',5,92)

    change_pracice_agreement(pre_df,post_df,'Q4',5,98)
    
    results.loc[99, 'Value'] = round(gender_discrimination(pre_df, 'Q5', 'Q6', 10), 2)
    results.loc[100, 'Value'] = round(gender_discrimination(post_df, 'Q5', 'Q6', 10),2)

    results.loc[101,'Value'] = round(change_gender_discrimination(pre_df, post_df, 'Q5', 'Q6',10), 2)
    results.loc[102,'Value'] = round(change_gender_discrimination(pre_df_women_only, post_df_women_only, 'Q5', 'Q6',10), 2)
    results.loc[103,'Value'] = round(change_gender_discrimination(pre_df_men_only, post_df_men_only, 'Q5', 'Q6',10), 2)
    
    results.loc[104,'Value'] = round(change_gender_discrimination(pre_df_general, post_df_general, 'Q5', 'Q6',10), 2)
    results.loc[105,'Value'] = round(change_gender_discrimination(pre_df_gender_specific, post_df_gender_specific, 'Q5', 'Q6',10), 2)
    
    return results

Now we must define the various calculation functions we have used above to perform our analysis.

In [126]:
def change_supervisory_responsibility(pre_df, post_df, question):
   pre_supervisory_responsibility = float(0)
   post_supervisory_responsibility = float(0)
   for x in pre_df.index:
      if float(pre_df.loc[x, question]) == float(1): 
         pre_supervisory_responsibility = pre_supervisory_responsibility + 1
   for x in post_df.index:
      if float(post_df.loc[x, question]) == float(1): 
         post_supervisory_responsibility = post_supervisory_responsibility + 1
   if pre_supervisory_responsibility == float(0):
      return float(0)
   if post_supervisory_responsibility == float(0):
      return float(0)
   change = 100*((post_supervisory_responsibility/len(post_df.index))/(pre_supervisory_responsibility/len(pre_df.index))-1)
   return change

In [127]:
def count(df, question, domain_max, start_index):
    for x in range(1, domain_max+1):
        count=0
        for y in df.index:
            if float(df.loc[y, question + '_' + str(x)]) == float(1):
                count = count + 1
        count = 100*(count/len(df.index))
        results.loc[start_index+x, 'Value'] = round(count, 2)
    return count

In [128]:
def change_correct_count(pre_df, post_df, question, domain_max):
    change_counting_var = float(0)
    for x in range(1, domain_max+1):
        pre_adjusted_count = float(0)
        post_adjusted_count = float(0)
        pre_count=0
        for y in pre_df.index:
            if float(pre_df.loc[y, question + '_' + str(x)]) == float(1):
                pre_count = pre_count + 1
        pre_adjusted_count = pre_count/len(pre_df.index)
        post_count=0
        for y in post_df.index:
            if float(post_df.loc[y, question + '_' + str(x)]) == float(1):
                post_count = post_count + 1
        post_adjusted_count = post_count/len(post_df.index)
        if pre_adjusted_count != 0:
            change_counting_var = change_counting_var + abs(post_adjusted_count/pre_adjusted_count-1)
    change = 100*(change_counting_var/domain_max)
    return change

In [129]:
def change_harrassment_recognition(pre_df, post_df, question, domain_min, domain_max, start_index, average_true_false):
    change_counting_var = float(0)
    change = float(0)
    average_change = float(0)
    for x in range(domain_min, domain_max+1):
        pre_count= pre_df[str(question + '_' + str(x))].astype('int').mean()
        post_count=post_df[str(question + '_' + str(x))].astype('int').mean()
        if pre_count != 0:
            change = 100*(post_count/pre_count - 1)
        else:
            change = 0
        change_counting_var = change_counting_var + change
        if average_true_false == False:
            results.loc[start_index+x, 'Value'] = round(change,2)    
    if average_true_false == True:
        average_change = change_counting_var/domain_max
        results.loc[start_index, 'Value'] = round(average_change, 2)
    return average_change

In [130]:
def practice_agreement(df,question,subquestion_count,start_index_exclusive):
        for x in range(1,subquestion_count+1):
            average_agreement = df[str(question + '_' + str(x))].astype('int').mean()
            results.loc[start_index_exclusive + x, 'Value'] = round(average_agreement,2)
        return average_agreement



In [131]:
def change_pracice_agreement(pre_df, post_df, question,subquestion_count,start_index_exclusive):
    counting_var = float(0)
    for x in range(1,subquestion_count+1):
        pre_average_agreement = pre_df[str(question + '_' + str(x))].astype('int').mean()
        post_average_agreement = post_df[str(question + '_' + str(x))].astype('int').mean()
        counting_var = counting_var + 100*abs((post_average_agreement/pre_average_agreement) - 1)
    change_average_agreement = counting_var/subquestion_count
    results.loc[start_index_exclusive,'Value'] = round(change_average_agreement,2)
    return change_average_agreement
        

In [132]:
def gender_discrimination(df,q_female, q_male, subquestion_count):
    counting_var = float(0)
    for x in range(1,subquestion_count+1):
        female_count = df[str(q_female + '_' + str(x))].astype('int').sum()
        male_count = df[str(q_male + '_' + str(x))].astype('int').sum()
        if male_count != int(0):
            counting_var = counting_var + 100*abs(female_count/male_count-1)
    overall_discrimination = counting_var/subquestion_count
    return overall_discrimination


In [133]:
def change_gender_discrimination(pre_df, post_df, q_female, q_male, subquestion_count):
    pre_discrimination = gender_discrimination(pre_df, q_female, q_male, subquestion_count)
    post_discrimination = gender_discrimination(post_df, q_female, q_male, subquestion_count)
    change = 100*(post_discrimination/pre_discrimination - 1)
    return change


Now we run the code!

In [134]:
report = analyze(clean(pre_sst_responses), clean(post_sst_responses))
report.to_csv('C:\\Users\\arira\\OneDrive\\Desktop\\Better Work\\Nicaragua Project\\CSV Files\\Report.csv', index=True)

